In [70]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from enum import Enum

class Softmax():
    @staticmethod
    def compute(x):
        x = np.clip(x, 1e-15, 1 - 1e-15)
        e_x = np.exp(x - np.max(x, axis=0, keepdims=True))
        return e_x / np.sum(e_x, axis=0, keepdims=True)
    
    def compute_derivative(self, x):
        value = self.compute(x)
        return value * (1 - value)
    
class ReLU():
    @staticmethod
    def compute(x):
        return np.maximum(x, 0)
    
    @staticmethod
    def compute_derivative(x):
        return x > 0
    
class LeakyReLU:
    @staticmethod
    def compute(x, alpha=0.01):
        return np.where(x >= 0, x, alpha * x)

    @staticmethod
    def compute_derivative(x, alpha=0.01):
        return np.where(x >= 0, 1, alpha)
    
class Sigmoid():
    @staticmethod
    def compute(x):
        return 1 / (1 + np.exp(-x))

    def compute_derivative(self, x):
        return x * (1 - x)
    
class CrossEntropy():
    @staticmethod
    def compute(y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return - (y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred)).mean()

    @staticmethod
    def compute_derivative(y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return - (y / y_pred) + (1 - y) / (1 - y_pred)
    
class MSE:
    @staticmethod
    def compute(y, y_pred):
        return ((y - y_pred) ** 2).mean()
    
    @staticmethod
    def compute_derivative(y, y_pred):
        return -2*(y - y_pred) / y.shape[0]
    
class Tanh:
    @staticmethod
    def compute(x):
        return np.tanh(x)

    @staticmethod
    def compute_derivative(x):
        return 1 - x ** 2
    
class Linear:
    @staticmethod
    def compute(x):
        return x

    @staticmethod
    def compute_derivative(x):
        return np.ones_like(x)

class NeuralNetworkStructure:
    def __init__(self, inputSize, outputSize, hiddenLayerSizes, hiddenLayerFunction, outputLayerFunction):
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenLayerSizes = hiddenLayerSizes
        self.layersSizes = hiddenLayerSizes + [outputSize]
        self.activationFunction = [hiddenLayerFunction] * len(hiddenLayerSizes) + [outputLayerFunction]
        self.layerInput = [None] * len(self.layersSizes)
        self.layerOutput = [None] * len(self.layersSizes)

        self.initializeWeights()

    def initializeWeights(self):
        self.weights = []
        self.bias = []
        
        previousLayerSize = self.inputSize
        for layerSize in self.layersSizes:
            self.weights.append(np.random.rand(layerSize, previousLayerSize) - 0.5)
            self.bias.append(np.random.rand(layerSize, 1) - 0.5)
            previousLayerSize = layerSize
            

class NeuralNetwork:
    def __init__(self, neuralNetworkStructure, epochs, learningRate, lossFunction):
        self.structure = neuralNetworkStructure
        self.lossFunction = lossFunction
        self.learningRate = learningRate
        self.epochs = epochs
        
    
    def Forward(self, X):
        previous_layer = np.reshape(X, (X.shape[0], 1))
        for id in range(len(self.structure.layersSizes)):
            self.structure.layerInput[id] = self.structure.weights[id].dot(previous_layer) + self.structure.bias[id]
            self.structure.layerOutput[id] = self.structure.activationFunction[id].compute(self.structure.layerInput[id])
            previous_layer = self.structure.layerOutput[id]
        return previous_layer
    
    
    def Backward(self, X, ExpectedY, PredictedY):
        previous_layer_error = self.lossFunction.compute_derivative(ExpectedY, PredictedY)
        
        for id in range(len(self.structure.layersSizes) -1, -1, -1):
            previous_layer_output = self.structure.layerOutput[id - 1] if id != 0 else X
            
            delta = previous_layer_error * self.structure.activationFunction[id].compute_derivative(self.structure.layerOutput[id])
            deltaW = np.dot(delta, previous_layer_output.T)
            deltaB = delta
            previous_layer_error = np.dot(self.structure.weights[id].T, delta)
                                        
            self.structure.weights[id] -= self.learningRate * deltaW
            self.structure.bias[id] -= self.learningRate * deltaB
    
    
    def Train(self, X, ExpectedY):
    
        X = np.reshape(X, (X.shape[0], 1))
        ExpectedY = np.reshape(ExpectedY, (ExpectedY.shape[0], 1))
    
        predictedY = self.Forward(X)
        self.Backward(X, ExpectedY, predictedY)
            

    def Test(self, train_inputs, train_results, test_inputs, test_results):
        
        for epoch in range(self.epochs):
            for i in range(len(train_inputs)):
              self.Train(train_inputs[i], train_results[i])
            
            correct = 0
            for i in range(len(test_inputs)):
                predictedY = self.Forward(np.reshape(test_inputs[i], (-1, 1)))

                if np.argmax(predictedY) == np.argmax(test_results[i]):
                    correct += 1
            
            test_accuracy = correct / len(test_inputs)
            print(f"Epoch {epoch + 1}/{self.epochs}, Test Accuracy: {test_accuracy * 100:.2f}% Correct: {correct}, All: {len(test_inputs)} ")            
    
    
    
    
#class NeuralNetworkClassificator(NeuralNetwork): 
    
    
    
#class NeuralNetworkRegressor(NeuralNetwork): 


In [77]:
import numpy as np
import pandas as pd

test = pd.read_csv("classification/data.simple.test.100.csv", sep=",")
train = pd.read_csv("classification/data.simple.train.100.csv", sep=",")

train_vectors = np.array(train[["x", "y"]])
train_results = np.array(train["cls"] - 1)
test_vectors = np.array(test[["x", "y"]])
test_results = np.array(test["cls"] - 1)

num_classes = 2
train_results = np.eye(num_classes)[train_results]
test_results = np.eye(num_classes)[test_results]

nnS = NeuralNetworkStructure(
    inputSize = 2, 
    outputSize = 2, 
    hiddenLayerSizes = [8], 
    hiddenLayerFunction = Sigmoid(), 
    outputLayerFunction = Softmax())

nn = NeuralNetwork(  
    epochs = 100, 
    learningRate = 0.01,
    neuralNetworkStructure = nnS,
    lossFunction = CrossEntropy())

nn.Test(train_vectors, train_results, test_vectors, test_results)

Epoch 1/100, Test Accuracy: 53.00% Correct: 53, All: 100 
Epoch 2/100, Test Accuracy: 86.00% Correct: 86, All: 100 
Epoch 3/100, Test Accuracy: 92.00% Correct: 92, All: 100 
Epoch 4/100, Test Accuracy: 93.00% Correct: 93, All: 100 
Epoch 5/100, Test Accuracy: 95.00% Correct: 95, All: 100 
Epoch 6/100, Test Accuracy: 96.00% Correct: 96, All: 100 
Epoch 7/100, Test Accuracy: 97.00% Correct: 97, All: 100 
Epoch 8/100, Test Accuracy: 99.00% Correct: 99, All: 100 
Epoch 9/100, Test Accuracy: 100.00% Correct: 100, All: 100 
Epoch 10/100, Test Accuracy: 100.00% Correct: 100, All: 100 
Epoch 11/100, Test Accuracy: 100.00% Correct: 100, All: 100 
Epoch 12/100, Test Accuracy: 100.00% Correct: 100, All: 100 
Epoch 13/100, Test Accuracy: 100.00% Correct: 100, All: 100 
Epoch 14/100, Test Accuracy: 99.00% Correct: 99, All: 100 
Epoch 15/100, Test Accuracy: 98.00% Correct: 98, All: 100 
Epoch 16/100, Test Accuracy: 98.00% Correct: 98, All: 100 
Epoch 17/100, Test Accuracy: 98.00% Correct: 98, All: 1